# 🔬 Préparation des récepteurs pour le Docking avec Smina

Ce notebook télécharge les récepteurs biomoléculaires (PDB), les convertit au format `.pdbqt`, et vérifie la qualité de conversion.

In [1]:
# ✅ Étape 1 : Installation d’Open Babel
!apt-get update
!apt-get install -y openbabel wget

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
# ✅ Étape 2 : Téléchargement des fichiers PDB depuis RCSB
!mkdir -p receptors && cd receptors && \
wget https://files.rcsb.org/download/1DMP.pdb && \
wget https://files.rcsb.org/download/2XJX.pdb && \
wget https://files.rcsb.org/download/7PG9.pdb && \
wget https://files.rcsb.org/download/5JWA.pdb

--2025-06-26 19:46:50--  https://files.rcsb.org/download/1DMP.pdb
Resolving files.rcsb.org (files.rcsb.org)... 3.162.140.107, 3.162.140.122, 3.162.140.76, ...
Connecting to files.rcsb.org (files.rcsb.org)|3.162.140.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘1DMP.pdb.1’

1DMP.pdb.1              [  <=>               ] 186.44K   455KB/s    in 0.4s    

2025-06-26 19:46:57 (455 KB/s) - ‘1DMP.pdb.1’ saved [190917]

--2025-06-26 19:46:57--  https://files.rcsb.org/download/2XJX.pdb
Resolving files.rcsb.org (files.rcsb.org)... 3.162.140.24, 3.162.140.107, 3.162.140.76, ...
Connecting to files.rcsb.org (files.rcsb.org)|3.162.140.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘2XJX.pdb.1’

2XJX.pdb.1              [  <=>               ] 202.74K   670KB/s    in 0.3s    

2025-06-26 19:46:59 (670 KB/s) - ‘2XJX.pdb.1’ saved [207603]

--

In [3]:
# ✅ Étape 3 : Conversion en .pdbqt avec Open Babel
import os

targets = ["1DMP", "2XJX", "7PG9", "5JWA"]
errors = []

for pdb in targets:
    input_pdb = f"receptors/{pdb}.pdb"
    output_pdbqt = f"receptors/{pdb}.pdbqt"
    print(f"🔄 Conversion de {pdb} ...")
    ret = os.system(f"obabel {input_pdb} -O {output_pdbqt} --partialcharge gasteiger")
    if ret != 0 or not os.path.exists(output_pdbqt):
        errors.append(pdb)

if errors:
    print(f"❌ Conversion échouée pour : {errors}")
else:
    print("✅ Tous les fichiers ont été convertis avec succès.")

🔄 Conversion de 1DMP ...


1 molecule converted


🔄 Conversion de 2XJX ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is receptors/2XJX.pdb)

1 molecule converted


🔄 Conversion de 7PG9 ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is receptors/7PG9.pdb)

1 molecule converted


🔄 Conversion de 5JWA ...
✅ Tous les fichiers ont été convertis avec succès.


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is receptors/5JWA.pdb)

0 molecules converted


In [4]:
# ✅ Étape 4 : Vérification de la présence et taille des fichiers .pdbqt
import os

for pdb in targets:
    path = f"receptors/{pdb}.pdbqt"
    if os.path.exists(path):
        size = os.path.getsize(path) / 1024
        print(f"✔️ {pdb}.pdbqt — {size:.1f} Ko")
    else:
        print(f"⚠️ Fichier manquant : {pdb}.pdbqt")

✔️ 1DMP.pdbqt — 291.7 Ko
✔️ 2XJX.pdbqt — 9709.9 Ko
✔️ 7PG9.pdbqt — 36922.9 Ko
✔️ 5JWA.pdbqt — 0.0 Ko


In [5]:
# Exemple avec biopython
from Bio.PDB import PDBParser
import numpy as np

def get_center_of_receptor(pdb_file):
    parser = PDBParser()
    structure = parser.get_structure("receptor", pdb_file)
    coords = [atom.coord for atom in structure.get_atoms() if atom.element != "H"]
    center = np.mean(coords, axis=0)
    return center

x, y, z = get_center_of_receptor("receptors/1DMP.pdb")
print(f"--center_x {x:.1f} --center_y {y:.1f} --center_z {z:.1f}")


--center_x -11.8 --center_y 20.4 --center_z 27.9


In [1]:
from nbformat import v4, write
from pathlib import Path

# Génération d’un notebook avec les nouvelles cibles et téléchargement auto
cells = []

# 📦 Import + setup
cells.append(v4.new_code_cell("""\
import os
import time
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from Bio.PDB import PDBParser

# Téléchargement des PDB supplémentaires
os.makedirs("receptors", exist_ok=True)
pdb_ids = ["1DMP", "2XJX", "7PG9", "1SYH", "4LDE", "6Y2F"]
for pdb in pdb_ids:
    if not os.path.exists(f"receptors/{pdb}.pdb"):
        os.system(f"wget -O receptors/{pdb}.pdb https://files.rcsb.org/download/{pdb}.pdb")

def compute_box_from_pdb(pdb_path, padding=6.0):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("receptor", pdb_path)
    atoms = [atom.coord for atom in structure.get_atoms() if atom.element != "H"]
    coords = np.array(atoms)
    min_corner = coords.min(axis=0)
    max_corner = coords.max(axis=0)
    center = coords.mean(axis=0)
    size = (max_corner - min_corner) + padding
    return center, size

def extract_affinity_from_log(log_file):
    affinity, found = None, False
    try:
        with open(log_file, 'r') as f:
            for line in f:
                if found and re.match(r'^\\s*1\\s+-?\\d+\\.\\d+', line):
                    affinity = float(line.split()[1])
                    break
                if line.strip().startswith("mode |"):
                    found = True
    except Exception:
        pass
    return affinity

def run_docking_fallback(ligand_pdbqt, receptor_pdbqt, receptor_pdb, output_dir, try_tools=["smina", "vina"]):
    os.makedirs(output_dir, exist_ok=True)
    base = os.path.basename(ligand_pdbqt).replace('.pdbqt', '')
    output_file = os.path.join(output_dir, base + '_out.pdbqt')
    log_file = os.path.join(output_dir, base + '.log')
    
    center, size = compute_box_from_pdb(receptor_pdb)
    cx, cy, cz = center
    sx, sy, sz = size

    for tool in try_tools:
        cmd = f"{tool} --receptor {receptor_pdbqt} --ligand {ligand_pdbqt} " \\
              f"--center_x {cx:.2f} --center_y {cy:.2f} --center_z {cz:.2f} " \\
              f"--size_x {sx:.1f} --size_y {sy:.1f} --size_z {sz:.1f} " \\
              f"--out {output_file} --log {log_file} --exhaustiveness 8"
        t0 = time.time()
        os.system(cmd)
        duration = time.time() - t0

        affinity = extract_affinity_from_log(log_file)
        if affinity is not None:
            print(f"✅ {tool.upper()} réussi pour {base} ({affinity:.2f} kcal/mol, {duration:.1f}s)")
            return affinity, tool
        else:
            print(f"⚠️ {tool} a échoué pour {base}")
    return None, "Failed"
"""))

# Boucle de docking avec toutes les cibles
cells.append(v4.new_code_cell("""\
ALL_LIGANDS_DIR = "ligands_pdbqt"
RECEPTORS = {
    "HIV1protease": ("receptors/1DMP.pdbqt", "receptors/1DMP.pdb"),
    "Hsp90":        ("receptors/2XJX.pdbqt", "receptors/2XJX.pdb"),
    "Proteasome":   ("receptors/7PG9.pdbqt", "receptors/7PG9.pdb"),
    "Neurodegenerative (1SYH)": ("receptors/1SYH.pdbqt", "receptors/1SYH.pdb"),
    "Malaria COPD (4LDE)": ("receptors/4LDE.pdbqt", "receptors/4LDE.pdb"),
    "COVID-19 (6Y2F)": ("receptors/6Y2F.pdbqt", "receptors/6Y2F.pdb")
}

ligands = [f for f in os.listdir(ALL_LIGANDS_DIR) if f.endswith(".pdbqt")]
results = []

for lig in tqdm(ligands, desc="Docking total"):
    lig_path = os.path.join(ALL_LIGANDS_DIR, lig)
    result_row = {"Molecule": lig}
    for target, (rec_pdbqt, rec_pdb) in RECEPTORS.items():
        out_dir = os.path.join("results", target)
        affinity, method = run_docking_fallback(lig_path, rec_pdbqt, rec_pdb, out_dir)
        result_row[target] = affinity
        result_row[target + "_method"] = method
    results.append(result_row)

df = pd.DataFrame(results)
df.to_csv("docking_matrix.csv", index=False)
df.head()"""))

# Visualisation
cells.append(v4.new_code_cell("""\
df = pd.read_csv("docking_matrix.csv")
aff_cols = [col for col in df.columns if col not in ["Molecule"] and not col.endswith("_method")]
df_long = df[["Molecule"] + aff_cols].melt(id_vars="Molecule", var_name="Target", value_name="Affinity")

plt.figure(figsize=(12, 6))
heatmap_data = df_long.pivot(index="Molecule", columns="Target", values="Affinity")
sns.heatmap(heatmap_data, cmap="vlag", center=0, annot=True, fmt=".1f")
plt.title("Carte de chaleur des affinités (kcal/mol)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()"""))

# Enregistrement du notebook
notebook = v4.new_notebook(cells=cells, metadata={"kernelspec": {"name": "python3", "display_name": "Python 3"}})
nb_path = Path("docking_all_targets_with_download.ipynb")
with open(nb_path, "w", encoding="utf-8") as f:
    write(notebook, f)

nb_path.as_posix()


'docking_all_targets_with_download.ipynb'